# --- Day 19: Aplenty ---

https://adventofcode.com/2023/day/19

## Parse the Input Data

In [6]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------

    """
    workflows = {}
    parts = []
    read_workflows = True

    with open(f'../inputs/{filename}.txt') as f:
        for line in f:
            # Skip blank line
            if line == "\n":
                read_workflows = False
                continue

            if read_workflows:
                flow_name = line[:line.find("{")]
                steps = line[line.find("{")+1:-2].split(",")

                workflows[flow_name] = []
                for step in steps:
                    rule = {}
                    if "<" in step or ">" in step or "=" in step:
                        rule["cat"] = step[0]
                        rule["op"] = step[1]
                        rule["val"] = int(step[2: step.find(":")])
                        rule["next"] = step[step.find(":")+1:]
                    else:
                        rule["cat"] = None
                        rule["op"] = None
                        rule["val"] = None
                        rule["next"] = step
                    workflows[flow_name].append(rule)

            # Read parts info
            else:
                part = {}
                cats = ["x", "m", "a", "s"]
                cat_vals = line.lstrip("{").rstrip("}\n").split(",")

                for c, v in zip(cats, cat_vals):
                    part[c] = int(v[2:])

                part["total"] = sum(part.values())

                parts.append(part)

    return workflows, parts

In [7]:
parse("test_workflows")

({'px': [{'cat': 'a', 'op': '<', 'val': 2006, 'next': 'qkq'},
   {'cat': 'm', 'op': '>', 'val': 2090, 'next': 'A'},
   {'cat': None, 'op': None, 'val': None, 'next': 'rfg'}],
  'pv': [{'cat': 'a', 'op': '>', 'val': 1716, 'next': 'R'},
   {'cat': None, 'op': None, 'val': None, 'next': 'A'}],
  'lnx': [{'cat': 'm', 'op': '>', 'val': 1548, 'next': 'A'},
   {'cat': None, 'op': None, 'val': None, 'next': 'A'}],
  'rfg': [{'cat': 's', 'op': '<', 'val': 537, 'next': 'gd'},
   {'cat': 'x', 'op': '>', 'val': 2440, 'next': 'R'},
   {'cat': None, 'op': None, 'val': None, 'next': 'A'}],
  'qs': [{'cat': 's', 'op': '>', 'val': 3448, 'next': 'A'},
   {'cat': None, 'op': None, 'val': None, 'next': 'lnx'}],
  'qkq': [{'cat': 'x', 'op': '<', 'val': 1416, 'next': 'A'},
   {'cat': None, 'op': None, 'val': None, 'next': 'crn'}],
  'crn': [{'cat': 'x', 'op': '>', 'val': 2662, 'next': 'A'},
   {'cat': None, 'op': None, 'val': None, 'next': 'R'}],
  'in': [{'cat': 's', 'op': '<', 'val': 1351, 'next': 'px'},


## Part 1
---

In [8]:
def check(left, op, right):
    if op == "<":
        return left.__lt__(right)
    elif op == ">":
        return left.__gt__(right)

In [9]:
def process(workflows, part, start):

    # base cases
    if start == "A":
        return part["total"]
    elif start == "R":
        return 0

    # process and recurse
    else:
        for rule in workflows[start]:
            if rule["op"] == None:
                return process(workflows, part, rule["next"])

            elif check(part[rule["cat"]], rule["op"], rule["val"]):
                return process(workflows, part, rule["next"])


In [10]:
def solve1(filename):
    workflows, parts = parse(filename)

    answer = 0
    for part in parts:
        answer += process(workflows, part, "in")

    return answer

### Run on Test Data

In [11]:
workflows, parts  = parse("test_workflows")
process(workflows, parts[0], "in") == 7540

True

In [12]:
solve1("test_workflows") == 19114

True

### Run on Input Data

In [13]:
solve1("workflows")

397061

## Part 2
---

In [14]:
import math

In [114]:
def process2(workflows, start, ranges={"x" : [1, 4000], "m" : [1, 4000], "a" : [1, 4000], "s" : [1, 4000]}):

    answer = 0

    # base cases
    if start == "A":
        # return total
        print(ranges)
        print(math.prod([len(range(*ranges[c])) + 1 for c in "xmas"]))
        combos = math.prod([len(range(*ranges[c])) + 1 for c in "xmas"])
        # ranges = {"x" : [1, 4000], "m" : [1, 4000], "a" : [1, 4000], "s" : [1, 4000]}
        return combos
    elif start == "R":
        return 0

    # process and recurse
    else:
        for rule in workflows[start]:
            if rule["op"] == None:
                answer += process2(workflows, rule["next"], ranges)

            else:
                if rule["op"] == "<":
                    ranges[rule["cat"]][1] = rule["val"] - 1
                    answer += process2(workflows, rule["next"], ranges)
                    # ranges={"x" : [1, 4000], "m" : [1, 4000], "a" : [1, 4000], "s" : [1, 4000]}

                elif rule["op"] == ">":
                    ranges[rule["cat"]][0] = rule["val"] + 1
                    answer += process2(workflows, rule["next"], ranges)
                    # ranges={"x" : [1, 4000], "m" : [1, 4000], "a" : [1, 4000], "s" : [1, 4000]}

    return answer
        # for rule in workflows[start]:
        #     if rule["op"] == None:
        #         process2(workflows, part_cat, rule["next"], total)

            # elif rule["op"] == "<":
            #     left = total * rule["val"] - 1
            #     right = total * (4000 - rule["val"] + 1)
            #     return process2(workflows, part_cat, rule["next"], left) #* process2(workflows, part_cat, rule["next"], right)

            # elif rule["op"] == ">":
            #     left = total * (4000 - rule["val"] + 1)
            #     right = total * (rule["val"] - 1)
            #     return process2(workflows, part_cat, rule["next"], left) #* process2(workflows, part_cat, rule["next"], right)

            # elif rule["op"] == "=":
            #     left = total
            #     right = total * 3999
            #     return process2(workflows, part_cat, rule["next"], left) #* process2(workflows, part_cat, rule["next"], right)

    # return total

In [115]:
from itertools import product

In [116]:
list(product([0, 4001], repeat=4))

[(0, 0, 0, 0),
 (0, 0, 0, 4001),
 (0, 0, 4001, 0),
 (0, 0, 4001, 4001),
 (0, 4001, 0, 0),
 (0, 4001, 0, 4001),
 (0, 4001, 4001, 0),
 (0, 4001, 4001, 4001),
 (4001, 0, 0, 0),
 (4001, 0, 0, 4001),
 (4001, 0, 4001, 0),
 (4001, 0, 4001, 4001),
 (4001, 4001, 0, 0),
 (4001, 4001, 0, 4001),
 (4001, 4001, 4001, 0),
 (4001, 4001, 4001, 4001)]

In [117]:
def solve2(filename):
    workflows, _ = parse(filename)

    # parts = []
    # for vals in product([0, 4001], repeat=4):
    #     parts.append(dict(zip("xmas", vals)))

    # print(parts)

    # answer = 0
    # for part in parts:
    #     print(process2(workflows, part, "in"))
    #     answer += process2(workflows, part, "in")

    answer = process2(workflows, "in")

    return answer

### Run on Test Data

In [118]:
solve2("test_workflows") # == 167_409_079_868_000

{'x': [1, 1415], 'm': [1, 4000], 'a': [1, 2005], 's': [1, 1350]}
15320205000000
{'x': [2663, 1415], 'm': [1, 4000], 'a': [1, 2005], 's': [1, 1350]}
10827000000
{'x': [2663, 1415], 'm': [2091, 4000], 'a': [1, 2005], 's': [1, 1350]}
5169892500
{'x': [2441, 1415], 'm': [2091, 4000], 'a': [3334, 2005], 's': [1, 536]}
1023760
{'x': [2441, 1415], 'm': [2091, 4000], 'a': [3334, 2005], 's': [3449, 536]}
1910
{'x': [2441, 1415], 'm': [1549, 4000], 'a': [3334, 2005], 's': [3449, 536]}
2452
{'x': [2441, 1415], 'm': [1549, 4000], 'a': [3334, 2005], 's': [3449, 536]}
2452
{'x': [2441, 1415], 'm': [839, 1800], 'a': [3334, 2005], 's': [3449, 536]}
962
{'x': [2441, 1415], 'm': [839, 1800], 'a': [1717, 2005], 's': [3449, 536]}
278018


15336203202054

In [19]:
3830051250 * 4000

15320205000000

[1,1415],[1,4000],[1,2005],[1,1350] 15320205000000

In [18]:
import math
math.prod([len(range(1415)), len(range(4000)), len(range(2005)), len(range(1350))])

15320205000000

### Run on Input Data

In [15]:
# solve2("workflows")